In [ ]:
import numpy as np
import cv2
import  imutils
import sys
import pytesseract
import pandas as pd
import time
from imutils.video import VideoStream
from imutils.video import FileVideoStream
import argparse
import datetime
import time
import IPython.display as ipd
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:

# Read the image file
#image = cv2.imread('Car.jpeg')
# initialize the first frame in the video stream
#vs = cv2.VideoCapture(r"C:\Users\saumya.m.tiwari\Downloads\People\opencv-car-number-plate-detection--master\video12.mp4")
vs = cv2.VideoCapture(r"C:\Users\saumya.m.tiwari\Downloads\merge.mp4")
firstFrame = None

# loop over the frames of the video
while True:
	# grab the current frame and initialize the occupied/unoccupied
	# text
    frame = vs.read()
    frame = frame[1]
    #text = "No Disturbance"

	# if the frame could not be grabbed, then we have reached the end
	# of the video
    if frame is None:
        break

    # resize the frame, convert it to grayscale, and blur it
    # Resize the image - change width to 500
    frame = imutils.resize(frame, width=500)

# Display the original image
    cv2.imshow("Original Frame", frame)

# RGB to Gray scale conversion
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#cv2.imshow("1 - Grayscale Conversion", gray)

# Noise removal with iterative bilateral filter(removes noise while preserving edges)
    gray = cv2.bilateralFilter(frame, 11, 17, 17)
#cv2.imshow("2 - Bilateral Filter", gray)

	# if the first frame is None, initialize it
    if firstFrame is None:
        firstFrame = gray
        continue




# Find Edges of the grayscale image
    edged = cv2.Canny(gray, 170, 200)
#cv2.imshow("4 - Canny Edges", edged)

# Find contours based on Edges
    (new, cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30] #sort contours based on their area keeping minimum required area as '30' (anything smaller than this will not be considered)
    NumberPlateCnt = None #we currently have no Number plate contour

# loop over our contours to find the best possible approximate contour of number plate
    count = 0
    for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            if len(approx) == 4:  # Select the contour with 4 corners
                NumberPlateCnt = approx #This is our approx Number Plate Contour
                break


# Drawing the selected contour on the original image
    cv2.drawContours(frame, NumberPlateCnt, -1, (0,255,0), 3)
    cv2.imshow("Final Image With Number Plate Detected", frame)

# Configuration for tesseract
    config = ('-l eng --oem 1 --psm 3')

# Run tesseract OCR on image
    text = pytesseract.image_to_string(frame, config=config)
    print(text)
    key = cv2.waitKey(1) & 0xFF

	# if the `q` key is pressed, break from the lop
    if key == ord("q"):
        break

cv2.waitKey(30) #Wait for user input before closing the images displayed
cv2.destroyAllWindows()